# AI-Based Traffic Management System (Jupyter Fixed Version)
This notebook works fully in Jupyter (no cv2.imshow or cv2.waitKey).

In [1]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import cv2

def show_frame_jupyter(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 5))
    plt.imshow(frame_rgb)
    plt.axis('off')
    clear_output(wait=True)
    display(plt.gcf())

In [3]:
config_content = """
MIN_VEHICLES = 2
MAX_VEHICLES = 20
NUM_LANES = 3
SMOOTH_WINDOW = 5
OUTPUT_LOG = 'signal_timings_log.csv'
"""

with open('config.py', 'w') as f:
    f.write(config_content)

print('config.py created!')

config.py created!


In [ ]:
import cv2
import numpy as np
from IPython.display import display, clear_output
from PIL import Image
import time

# -----------------------------
# Function to show image in Jupyter
# -----------------------------
def show_frame_jupyter(frame):
    clear_output(wait=True)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))   # <-- FIXED (will show image, not array)

# -----------------------------
# Open webcam
# -----------------------------
cap = cv2.VideoCapture(0)

bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=120, varThreshold=40)

print("▶ Running traffic detection... Stop using Kernel → Interrupt")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Webcam error")
        break

    frame = cv2.resize(frame, (900, 500))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    mask = bg_subtractor.apply(gray)
    _, thresh = cv2.threshold(mask, 180, 255, cv2.THRESH_BINARY)

    dilated = cv2.dilate(thresh, np.ones((7,7), np.uint8), iterations=2)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    vehicle_count = sum(1 for c in contours if cv2.contourArea(c) > 1500)

    green_time = max(5, min(20, vehicle_count * 2))

    cv2.putText(frame, f"Vehicles: {vehicle_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)

    cv2.putText(frame, f"Green Time: {green_time}s", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    show_frame_jupyter(frame)

    time.sleep(0.05)

cap.release()
print("Traffic monitoring stopped.")